# CV19SIM: EPIcSuite simulator interface

Perdón la pobresa, work in progress. Será enriquecido 

In [20]:
import numpy as np
import pandas as pd
import toml
import matplotlib.pyplot as plt
import time
from datetime import datetime
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator

import epyestim
import epyestim.covid19 as covid19
from epyestim.distributions import discretise_gamma
from datetime import timedelta

In [21]:
# rpy2 imports
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects
pandas2ri.activate()
# R library
global eps
eps = importr("EpiEstim")

In [22]:
#Pop-up plots
import platform
OS = platform.system()
if OS == 'Linux' or OS == 'Darwin':
    %matplotlib tk    
elif OS == 'Windows':
    %matplotlib qt    
import sys
from pathlib import Path
sys.path.insert(1, '../')

In [4]:
cd ..

/home/samuel/covid19geomodeller


In [5]:
from src2.models.cv19sim import cv19sim

# Funciones para cálculo de R

In [6]:
# Analitico para simulación
def R_analytic(sim):
    t = np.array(sim.t)
    beta = np.array([sim.beta(j) for j in t])
    S = np.array(sim.S)
    tI_R = np.array([sim.tI_R(j) for j in t])
    N = sim.N
    return beta*S*tI_R/N

In [7]:
# Felipe
def R_effective(smoothed, window):
    df = pd.DataFrame({"dates": smoothed.index.tolist(), "cases": smoothed.values})
    rdf = pandas2ri.py2ri(df)
    r = robjects.r
    window = window  # always 5, test
    start = r.seq(2, df.values.shape[0]-window)
    end = r.seq( 2+ window,df.values.shape[0])
    results = eps.estimate_R(rdf[1], method="parametric_si", config = 
                              eps.make_config(t_start = start,
                               t_end = end,
                               mean_si = 4.03,
                               std_si = np.sqrt(6.5/(0.62**2))))
    results2 = dict(results.items())
    rhat = pandas2ri.ri2py(results2["R"])
    rhat.index = smoothed.index[2+window-1:]
    return rhat

In [8]:
# Felipe
def R_effectivegamma(smoothed, window):
    df = pd.DataFrame({"dates": smoothed.index.tolist(), "cases": smoothed.values})
    rdf = pandas2ri.py2ri(df)
    r = robjects.r
    window = window  # always 5, test
    start = r.seq(2, df.values.shape[0]-window)
    end = r.seq( 2+ window,df.values.shape[0])
    results = eps.estimate_R(rdf[1], method="parametric_si", config = 
                              eps.make_config(si_parametric_distr = "L",t_start = start,
                               t_end = end,
                               mean_si = 5,
                               std_si = 2))
    results2 = dict(results.items())
    rhat = pandas2ri.ri2py(results2["R"])
    rhat.index = smoothed.index[2+window-1:]
    return rhat

## Generación de datos sintéticos

In [9]:
cfgfile = 'config_files/SEIR.toml'
cfg = toml.load(cfgfile) # no es necesario, pero es util para ver el archivo de configuracion

### Creación objeto de multisimulación

In [10]:
beta = 0.5
#beta = '{"function":"Events","values":[0.3,0.3,0.3],"days":[[0,135],[135,190],[190,500]]}'
tI_R = 3
tE_I = 3
population = 100000
I = 100
I_d = 20
R = 0
mu = 1.5

In [11]:
sims = cv19sim(cfg,beta=beta,tI_R=tI_R,tE_I=tE_I,R=R,population=population,I_det=I, I_d_det =I_d,mu=mu,t_end=500)

In [12]:
# Resolver EDOS
sims.integrate()

In [13]:
sim = sims.sims[0]

In [14]:
#plt.plot(sim.t,sim.S,label='S')
#plt.plot(sim.t,sim.E,label='E')
plt.plot(sim.t,sim.I_d,label='I')
#plt.plot(sim.t,sim.R,label='R')
plt.legend(loc=0)

# Cálculo de R

## 1. Solución Analítica
$$
  R_e(t) = \beta(t)\frac{S(t)}{N}t_{IR}
$$

In [15]:
initdate = datetime(2020,3,1)
dates = [initdate+timedelta(days=int(j)) for j in sim.t]

In [16]:
cases = pd.Series(data=sim.I_d,index=dates)
cases_ac = pd.Series(data=sim.I_ac,index=dates)

In [17]:
R_felipe95 = R_effective(cases,7)['Quantile.0.95(R)']
R_felipe25 = R_effective(cases,7)['Quantile.0.25(R)']

In [18]:
Ra = R_analytic(sim)
R_felipe = R_effective(cases,7)['Mean(R)']
R_felipegamma = R_effectivegamma(cases,7)['Mean(R)']
R_epistim = covid19.r_covid(cases[:-200],delay_distribution=np.array([1]),n_samples=300)['R_mean']

In [19]:
fig, axs2 = plt.subplots(figsize=(10,6),linewidth=5,edgecolor='black',facecolor="white")

axs2.set_ylabel('R',color='tab:red')
axs2.plot(sim.t,Ra,color='tab:blue',label='Analytic')
#axs2.plot(sim.t[:len(R_felipe25)],R_felipe95,alpha=0.5,color='tab:red',label='Felipestim',linestyle='dashed')
#axs2.plot(sim.t[:len(R_felipe95)],R_felipe25,alpha=0.5,color='tab:red',label='Felipestim',linestyle='dashed')

axs2.plot(sim.t[:len(R_felipe)],R_felipe,color='tab:red',label='Felipestim',linestyle='dashed')
#axs2.plot(sim.t[:len(R_felipe)],R_felipegamma,color='tab:green',label='Felipestimgamma',linestyle='dashed')
axs2.plot(sim.t[:len(R_epistim)],R_epistim,color='black',label='Epyestim',linestyle='dashed')
axs2.tick_params(axis='y', labelcolor='tab:red')
axs2.axhline(1,color='grey',linestyle='dotted')

axs = axs2.twinx()
axs.plot(sim.t,sim.I_d,color='tab:blue',label='Diarios',linestyle='solid')
#axs.plot(sim.t,sim.I,color='tab:blue',label='Activos')
axs.set_ylabel('Infectados',color='tab:blue')
axs.tick_params(axis='y', labelcolor='tab:blue')

#axs2.set_xlim(0,200)
fig.suptitle('Infectados y R')
axs.legend(loc=1)
axs2.legend(loc=2)
fig.show()


In [88]:
from epyestim.distributions import discretise_gamma